# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python37-gpu\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [2]:
#unbalancedのubUnderを使った例
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

In [3]:
# df_recipt の custom_id で amount の合計でgroupby()
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_tmp.head()

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [4]:
# df_customer と customer_id をキーとして merge()
df_tmp = pd.merge(df_customer, df_tmp, how='left', on='customer_id')
df_tmp.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,NaN
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,5088.0
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,NaN
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,875.0


In [5]:
# amount が NaN に buy_flg に 0 を入れる。それ以外は 1 (amount に数字があると 1)
df_tmp['buy_flg'] = df_tmp['amount'].apply(lambda x: 0 if np.isnan(x) else 1)
df_tmp.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,amount,buy_flg
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,NaN,0
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,NaN,0
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,5088.0,1
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,NaN,0
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,875.0,1


In [6]:
print('0の件数', len(df_tmp.query('buy_flg == 0')))
print('1の件数', len(df_tmp.query('buy_flg == 1')))

0の件数 13665
1の件数 8306


In [7]:
# RandomUnderSampler() を実行
# https://qiita.com/strg-frontier/items/e411d8007fce68f230d6
rs = RandomUnderSampler(random_state=71)
df_sample = rs.fit_sample(df_tmp, df_tmp.buy_flg)
df_sample

(          customer_id customer_name  gender_cd gender   birth_day  age  \
 0      CS002513000098         紺野 美帆          1     女性  1962-02-25   57   
 1      CS013403000031          宮部 明          0     男性  1973-04-07   45   
 2      CS023712000022        堀井 しほり          9     不明  1939-09-16   79   
 3      CS001613000071        宍戸 麻由子          1     女性  1956-06-16   62   
 4      CS039511000070         河野 佳乃          1     女性  1968-01-03   51   
 ...               ...           ...        ...    ...         ...  ...   
 16607  CS002513000310         柳田 美嘉          1     女性  1963-09-17   55   
 16608  CS005415000024          若林 花          1     女性  1969-03-03   50   
 16609  CS001413000156        上原 恵望子          1     女性  1974-12-10   44   
 16610  CS040314000065         谷川 紗季          1     女性  1987-01-14   32   
 16611  CS029414000065         上村 怜奈          1     女性  1970-10-19   48   
 
       postal_cd                 address application_store_cd  \
 0      185-0011     東京都国分寺市本多***

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [8]:
# 非正規形の表から、繰り返し部分が取り除かれたものが第1正規形
# 第1正規形の表から、部分関数従属している列が切り出されたものが第2正規形。
# 第2正規形の表から、主キー以外の列に関数従属している列が切り出されたものが第3正規形
# http://ext-web.edu.sgu.ac.jp/HIKO/Prog03/SenpaiKyozai/shiohara/formalize.html
# https://breezegroup.co.jp/202005/database-normalization/

df_gender = df_customer[['gender_cd', 'gender']].drop_duplicates()
df_customer_s = df_customer.drop(columns='gender')
df_customer_s.head()

,customer_id,customer_name,gender_cd,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
0,CS021313000114,大野 あや子,1,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0
1,CS037613000071,六角 雅彦,9,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
2,CS031415000172,宇多田 貴美子,1,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
3,CS028811000001,堀井 かおり,1,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
4,CS001215000145,田崎 美紀,1,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [9]:
df_product.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,P040101001,4,401,40101,198.0,149.0
1,P040101002,4,401,40101,218.0,164.0
2,P040101003,4,401,40101,230.0,173.0
3,P040101004,4,401,40101,248.0,186.0
4,P040101005,4,401,40101,268.0,201.0


In [10]:
df_category.head()

,category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
0,4,惣菜,401,御飯類,40101,弁当類
1,4,惣菜,401,御飯類,40102,寿司類
2,4,惣菜,402,佃煮類,40201,魚介佃煮類
3,4,惣菜,402,佃煮類,40202,海草佃煮類
4,4,惣菜,402,佃煮類,40203,野菜佃煮類


In [11]:
# df_product と df_categoryのカテゴリーコードだけを category_small_cd をキーとして merge()
df_product_full = pd.merge(df_product, df_category[['category_small_cd', 
                                                    'category_major_name',
                                                    'category_medium_name',
                                                    'category_small_name']], 
                           how = 'inner', on = 'category_small_cd')
df_product_full.head()

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類


---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [12]:
# encoding='UTF-8'
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.csv', encoding='UTF-8', index=False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [13]:
# encoding='CP932' 日本語シフトJIS のこと
df_product_full.to_csv('./data/P_df_product_full_CP932_header.csv', encoding='CP932', index=False)

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [14]:
# encoding='UTF-8'
df_product_full.to_csv('./data/P_df_product_full_UTF-8_noh.csv', header=False ,encoding='UTF-8', index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [15]:
df_tmp = pd.read_csv('./data/P_df_product_full_UTF-8_header.csv')
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [16]:
# header=None
df_tmp = pd.read_csv('./data/P_df_product_full_UTF-8_noh.csv', header=None)
df_tmp.head(10)

,0,1,2,3,4,5,6,7,8
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類


---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [17]:
# TSV（タブ区切り）にするには sep='\t' 
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.tsv', sep='\t', encoding='UTF-8', index=False)

---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [18]:
# pd.read_table() を使用
df_tmp = pd.read_table('./data/P_df_product_full_UTF-8_header.tsv', encoding='UTF-8')
df_tmp.head(10)

,product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_major_name,category_medium_name,category_small_name
0,P040101001,4,401,40101,198.0,149.0,惣菜,御飯類,弁当類
1,P040101002,4,401,40101,218.0,164.0,惣菜,御飯類,弁当類
2,P040101003,4,401,40101,230.0,173.0,惣菜,御飯類,弁当類
3,P040101004,4,401,40101,248.0,186.0,惣菜,御飯類,弁当類
4,P040101005,4,401,40101,268.0,201.0,惣菜,御飯類,弁当類
5,P040101006,4,401,40101,298.0,224.0,惣菜,御飯類,弁当類
6,P040101007,4,401,40101,338.0,254.0,惣菜,御飯類,弁当類
7,P040101008,4,401,40101,420.0,315.0,惣菜,御飯類,弁当類
8,P040101009,4,401,40101,498.0,374.0,惣菜,御飯類,弁当類
9,P040101010,4,401,40101,580.0,435.0,惣菜,御飯類,弁当類
